<h1>CNN-H1 4-19-2016</h1>

<strong>Abstract</strong>
Implementing the CNN-H1 using NN2 described in the paper: http://arxiv.org/pdf/1509.00244v1.pdf. 

<strong>Improvements</strong>
<ul>
<li>Narrowed the width of faces to focus on the face, cut out background</li>
<li>Validation split of 15%</li>
<li>fc6 layer has a length of 256</li>
</ul>

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import os
import fnmatch
import numpy as np

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

np.random.seed(123456)

Using Theano backend.
Using gpu device 1: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)


In [3]:
data_path = '../data/lfw_cropped'

img_rows_load, img_cols_load = 160, 160
img_rows, img_cols = 160, 120
num_people = 600

<h2>Loading Files</h2>

In [4]:
def get_face_to_file_path_dict():
    face_to_file_paths_dict = {}
    
    for root, dirnames, filenames in os.walk(data_path):
        for dirname in dirnames:
            if dirname not in face_to_file_paths_dict:
                face_to_file_paths_dict[dirname] = []
            directory_path = os.path.join(data_path, dirname)
            for filename in os.listdir(directory_path):
                if filename.endswith(".jpg"):
                    face_to_file_paths_dict[dirname].append(os.path.join(directory_path, filename))
                            
    return face_to_file_paths_dict

In [5]:
def get_face_to_file_paths_descending_list(face_to_file_paths_dict):
    return sorted(face_to_file_paths_dict.items(), key=lambda x: len(x[1]), reverse=True)

In [6]:
face_to_file_paths_dict = get_face_to_file_path_dict()

In [7]:
face_to_file_paths_list = get_face_to_file_paths_descending_list(face_to_file_paths_dict)[:num_people]

<h2>Data Pre-Processing</h2>

In [8]:
def image_read(f):
    return resize(rgb2grey(io.imread(f)), (img_rows_load, img_cols_load))

def reflection(image):
    return np.array([list(reversed(row)) for row in image])

def partition(image, top_left, rows, cols):
    return np.array([row[top_left[1]:top_left[1] + cols] for row in image[top_left[0]:top_left[0] + rows]])

In [9]:
images_by_class = [[image_read(f) for f in x[1]] for x in face_to_file_paths_list]

In [10]:
for i in range(len(images_by_class)):
    images_by_class[i] += [reflection(im) for im in images_by_class[i]]

In [11]:
for i in range(len(images_by_class)):
    images_by_class[i] = [partition(im, (0, 20), img_rows, img_cols) for im in images_by_class[i]]

In [12]:
X_train = np.array([image for images in images_by_class for image in images])
y_train = np.array([images[0] for images in enumerate(images_by_class) for image in images[1]])

In [13]:
zipped = np.array(zip(X_train, y_train))
np.random.shuffle(zipped)

In [14]:
X_train = np.array([x[0] for x in zipped])
y_train = np.array([x[1] for x in zipped])

In [15]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
Y_train = np_utils.to_categorical(y_train, len(images_by_class))

<h2> Training and Validation</h2>

In [16]:
def NN2(input_shape, nb_classes, nb_fc6):
    model = Sequential()
    
    # Layer 1
    model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=input_shape))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Layer 3
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 4
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    #Layer 5
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3))
    model.add(ZeroPadding2D((1,1)))
    model.add(AveragePooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(nb_fc6))
    model.add(Dense(nb_classes, activation='softmax'))
   
    return model

In [17]:
input_shape = (1, img_rows, img_cols)
nb_fc6 = 256

In [18]:
model = NN2(input_shape, num_people, nb_fc6)
model.compile(loss='categorical_crossentropy', optimizer='sgd')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, 
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 11352 samples, validate on 2004 samples
Epoch 1/10
11352/11352 [==============================] - 354s - loss: 5.9833 - acc: 0.0729 - val_loss: 5.8514 - val_acc: 0.0853
Epoch 2/10
11352/11352 [==============================] - 354s - loss: 5.7909 - acc: 0.0780 - val_loss: 5.8237 - val_acc: 0.0853
Epoch 3/10
11352/11352 [==============================] - 354s - loss: 5.7555 - acc: 0.0778 - val_loss: 5.8065 - val_acc: 0.0853
Epoch 4/10
11352/11352 [==============================] - 354s - loss: 5.7381 - acc: 0.0781 - val_loss: 5.8015 - val_acc: 0.0853
Epoch 5/10
11352/11352 [==============================] - 354s - loss: 5.7151 - acc: 0.0779 - val_loss: 5.7545 - val_acc: 0.0853
Epoch 6/10
11352/11352 [==============================] - 354s - loss: 5.6453 - acc: 0.0810 - val_loss: 5.7003 - val_acc: 0.0878
Epoch 7/10
11352/11352 [==============================] - 354s - loss: 5.5302 - acc: 0.0900 - val_loss: 5.5485 - val_acc: 0.0988
Epoch 8/10
11352/11352 [========================

In [ ]:
json_string = model.to_json()
open('models/CNN-H1-fc6-256-10e.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-fc6-256-10e.h5', overwrite=True)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001))
model.fit(X_train, Y_train, batch_size=32, nb_epoch=100,
        show_accuracy=True, verbose=1, shuffle=True, validation_split=.15)

Train on 11352 samples, validate on 2004 samples
Epoch 1/100
11352/11352 [==============================] - 354s - loss: 3.6105 - acc: 0.3104 - val_loss: 3.9552 - val_acc: 0.2884
Epoch 2/100
11352/11352 [==============================] - 354s - loss: 3.4018 - acc: 0.3357 - val_loss: 3.8391 - val_acc: 0.3049
Epoch 3/100
11352/11352 [==============================] - 354s - loss: 3.2611 - acc: 0.3545 - val_loss: 3.7778 - val_acc: 0.3024
Epoch 4/100
11352/11352 [==============================] - 354s - loss: 3.1156 - acc: 0.3730 - val_loss: 3.6384 - val_acc: 0.3258
Epoch 5/100
11352/11352 [==============================] - 354s - loss: 2.9774 - acc: 0.3963 - val_loss: 3.5607 - val_acc: 0.3323
Epoch 6/100
11352/11352 [==============================] - 354s - loss: 2.8385 - acc: 0.4121 - val_loss: 3.4523 - val_acc: 0.3548
Epoch 7/100
11352/11352 [==============================] - 354s - loss: 2.7040 - acc: 0.4312 - val_loss: 3.3627 - val_acc: 0.3628
Epoch 8/100
11352/11352 [================

In [ ]:
json_string = model.to_json()
open('models/CNN-H1-fc6-256-110.json', 'w').write(json_string)
model.save_weights('models/CNN-H1-fc6-256-110e.h5', overwrite=True)